## Dictator Game Design
- if run into error saying 'cannot choose from empty sequence', reloading the notebook and rerunning the codes will solve the bug

In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import random
import pandas as pd
import numpy as np


In [2]:
rang = [x / 10.0 for x in range(0, 11)]

def plus_to_param(num_p = 0, num_n = 0, r = 0):
    param = (1+r)**num_p*(1-r)**num_n
    return param

def gen_type_lst(num_agent):
    n_f= round(num_agent * 0.48)
    n_m = num_agent - n_f
    lst_types = []
    types = ['ff','nf','fm','nm'] #feminist female, non-feminist female, feminist male, non-feminist male

    num_ff = round(n_f * 0.47) #feminist female; data from statista (self report bias)
    lst_ff = ['ff'][:]*num_ff
    lst_types += lst_ff

    num_nf = n_f - num_ff
    lst_nf = ['nf'][:]*num_nf
    lst_types += lst_nf

    num_fm = round(n_m * 0.31) #feminist male; data from statista
    lst_fm = ['fm'][:]*num_fm
    lst_types += lst_fm

    num_nm = n_m - num_fm
    lst_nm = ['nm'][:]*num_nm
    lst_types += lst_nm

    return lst_types

def gen_age_group(l):
    s = sum([0.237, 0.524,0.126])
    l_teen = round(l * 0.237 / s)
    l_mid = round(l * 0.524 / s)
    l_old = l - l_teen - l_mid #data from statistica; not super precise; normalized to exclude children

    ages = ['t'][:]*l_teen + ['m'][:]*l_mid + ['o'][:]*l_old
    return ages

def gen_propo_type(typ,other,bench, rate):
    w = 0
    propo = bench
    if typ.startswith('f') and len(typ) > 1:
        if typ == 'ff':
            if other.type == 'ff':
                propo = bench * plus_to_param(num_p = 2, r = rate)
            elif other.type == 'nf':
                propo = bench * plus_to_param(num_n = 1, r = rate)
            elif other.type == 'nm':
                propo = bench * plus_to_param(num_n = 1, r = rate)
            elif other.type == 'f':
                w = other.model.ff_percent + other.model.nf_percent
                if w!=0:
                    propo = other.model.ff_percent * bench * plus_to_param(num_p = 2, r = rate) /w + other.model.nf_percent * bench * plus_to_param(num_n = 1, r = rate)/w
            elif other.type == 'm':
                w = other.model.fm_percent + other.model.nm_percent
                if w!=0:
                    propo = other.model.fm_percent * bench/w + other.model.nm_percent * bench* plus_to_param(num_n = 1, r = rate)/w
            else:
                propo = bench

        elif typ == 'fm':
            if other.type == 'ff':
                propo = bench * plus_to_param(num_p = 1, r = rate)
            elif other.type == 'fm':
                propo = bench * plus_to_param(num_p = 1, r = rate)
            elif other.type == 'nf':
                propo = bench * plus_to_param(num_p = 0, r = rate)
            elif other.type == 'nm':
                propo = bench * plus_to_param(num_n = 2, r = rate)
            elif other.type == 'f':
                w = other.model.ff_percent + other.model.nf_percent
                if w!=0:
                    propo = other.model.ff_percent * bench * plus_to_param(num_p = 1, r = rate) /w + other.model.nf_percent * bench * plus_to_param(num_p = 0, r = rate)/w
            elif other.type == 'm':
                w = other.model.fm_percent + other.model.nm_percent
                if w!=0:
                    propo = other.model.fm_percent * bench * plus_to_param(num_p = 1, r = rate) /w + other.model.nm_percent * bench * plus_to_param(num_n = 2, r = rate) /w

    elif typ.startswith('n') and len(typ) > 1:
        if typ == 'nf':
            if other.type == 'ff':
                propo = bench * plus_to_param(num_n = 3, r = rate)
            elif other.type == 'fm':
                propo = bench * plus_to_param(num_n = 0.5, r = rate)
            elif other.type == 'nf':
                propo = bench * plus_to_param(num_p = 0, r = rate)
            elif other.type == 'nm':
                propo = bench * plus_to_param(num_p = 0.5, r = rate)
            elif other.type == 'f':
                w = other.model.ff_percent + other.model.nf_percent
                if w!=0:
                    propo = other.model.ff_percent * bench * plus_to_param(num_n = 3, r = rate) /w + other.model.nf_percent * bench * plus_to_param(num_p = 0, r = rate)/w
            elif other.type == 'm':
                w = other.model.fm_percent + other.model.nm_percent
                if w!=0:
                    propo = other.model.fm_percent * bench * plus_to_param(num_n = 0.5, r = rate) /w + other.model.nm_percent * bench * plus_to_param(num_p = 0.5, r = rate) /w
        elif typ == 'nm':
            if other.type == 'ff':
                propo = bench * plus_to_param(num_n = 1.5, r = rate)
            elif other.type == 'fm':
                propo = bench * plus_to_param(num_n = 1.5, r = rate)
            elif other.type == 'nf':
                propo = bench * plus_to_param(num_n = 0.5, r = rate)
            elif other.type == 'nm':
                propo = bench * plus_to_param(num_p = 1.5, r = rate)
            elif other.type == 'f':
                w = other.model.ff_percent + other.model.nf_percent
                if w!=0:
                    propo = other.model.ff_percent * bench * plus_to_param(num_n = 1.5, r = rate) /w + other.model.nf_percent * bench *plus_to_param(num_n = 0.5, r = rate) /w
            elif other.type == 'm':
                w = other.model.fm_percent + other.model.nm_percent
                if w!=0:
                    propo = other.model.fm_percent * bench *plus_to_param(num_n = 1.5, r = rate) /w + other.model.nm_percent * bench * plus_to_param(num_p = 1.5, r = rate) /w


    return propo

class Agent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        r = random.choice(self.model.index)
        rand = self.model.lst_types[r]
        self.type = rand
        self.init_type = rand
        self.age = self.model.ages[r]
        if self.unique_id not in self.model.giving_df['id']:
            if self.init_type == 'fm':
                self.wealth = 10
            elif self.init_type == 'ff':
                self.wealth = 8
            elif self.init_type == 'nm':
                self.wealth = 5
            else:
                self.wealth = 1
        else:
            df = self.model.giving_df
            self.wealth = list(df['wealth'][df['id'] == self.unique_id])[-1]
        self.model.index.remove(r)
        self.met = [self.unique_id]
        self.tax_rate = 0
        self.group_wealth_l = 0
        self.group_wealth_h = 0
        self.met_history = []


    def step(self):
        if len(self.model.giving_df) > 9 and list(self.model.giving_df['id'])[-1] != self.unique_id:
            self.met = [self.unique_id]

        while len(self.met) != round(len(self.model.schedule.agents) * self.model.percentage):
            if len([i for i in self.model.giving_df['id'] if i == self.unique_id]) >= 1:
                if self.model.tax == True:
                    self.tax()
                self.set_type()
                
                self.model.rate = self.model.init_rate*(1-self.model.fem_percent)*(self.model.nm_percent + self.model.nf_percent)/2
            self.group_wealth_l = sum([i.wealth for i in
                                       self.model.schedule.agents if
                                       i.type == self.type]) / len([i.wealth
                                                                    for i in self.model.schedule.agents
                                                                    if i.type == self.type])
            self.group_wealth_h = sum([i.wealth for i in
                                       self.model.schedule.agents if
                                       i.type[-1] == self.type[-1]]) / len([i
                                                                            for i in self.model.schedule.agents
                                                                            if i.type[-1] == self.type[-1]])
            other_agent = random.choice(self.model.schedule.agents)
            if other_agent.unique_id not in self.met:
                self.met_history.append(set([self.unique_id,other_agent.unique_id]))
                other_agent.met_history.append(set([self.unique_id,other_agent.unique_id]))
                propo = self.gen_proporation(other_agent,self.model.times,self.model.lb,self.model.ub, self.model.rate, self.model.tax_res)
                if propo > 1:
                    propo = 1
                giving = round(propo * self.model.pie)
                other_agent.wealth += giving
                self.wealth += (self.model.pie - giving)
                self.met.append(other_agent.unique_id)
                setting = str(self.model.percentage)+'|'+str(self.model.times)+'|'+str(self.model.lb)+','+str(self.model.ub)+\
                          '|'+str(self.model.init_rate)+'|'+str(self.model.tax_res) + '|' + str(self.model.fm_cost)
                self.model.giving_df.loc[len(self.model.giving_df.index)] = [self.model.round,setting,self.model.fem_percent,
                                                                             int(self.unique_id), int(self.wealth),
                                                                             self.age,self.group_wealth_l, self.group_wealth_h,
                                                                             self.init_type, self.type, other_agent.type,
                                                                             giving,propo, self.model.ff_percent, self.model.fm_percent,
                                                                             self.model.nf_percent,self.model.nm_percent]

    def set_type(self):
        typ = self.type
        if len(typ) == 2:
            if typ in ['ff','nf']:
                h_type = 'f'
                ol_type = [i for i in ['ff','nf'] if i != typ][0]
            else:
                h_type = 'm'
                ol_type = [i for i in ['fm','nm'] if i != typ][0]


            l_h = [i.wealth for i in self.model.schedule.agents if i.type[-1] == typ[-1]]
            avg_h = sum(l_h) / len(l_h)

            l_l = [i.wealth for i in self.model.schedule.agents if i.type == typ]
            if len(l_l) != 0:
                avg_l = sum(l_l) / len(l_l)
            else:
                avg_l = 0

            l_ol = [i.wealth for i in self.model.schedule.agents if i.type == ol_type]
            if len(l_ol) != 0:
                avg_ol = sum(l_ol) / len(l_ol)
            else:
                avg_ol = 0
                
            self.group_wealth_l = avg_l
            self.group_wealth_h = avg_h

            if typ == 'ff':
                avg_l = self.model.ff_cost*avg_l
                avg_ol = self.model.nf_cost*avg_ol
            elif typ == 'nf':
                avg_l = self.model.nf_cost*avg_l
                avg_ol = self.model.ff_cost*avg_ol
            elif typ == 'fm':
                avg_l = self.model.fm_cost*avg_l
                avg_ol = self.model.nm_cost*avg_ol
            elif typ == 'nm':
                avg_l = self.model.nm_cost*avg_l
                avg_ol = self.model.fm_cost*avg_ol

            l1 = [typ, h_type, ol_type]
            l2 = [avg_l, avg_h, avg_ol]

            self.type = [l1[i] for i in range(3) if l2[i] == max(l2)][0]

        else:
            l_type1 = 'f' + typ
            l_type2 = 'n' + typ

            l_h = [i.wealth for i in self.model.schedule.agents if i.type == typ]
            avg_h = sum(l_h) / len(l_h)

            l_l1 = [i.wealth for i in self.model.schedule.agents if i.type == l_type1]
            if len(l_l1) != 0:
                avg_l1 = sum(l_l1) / len(l_l1)
            else:
                avg_l1 = 0

            l_l2 = [i.wealth for i in self.model.schedule.agents if i.type == l_type2]
            if len(l_l2) != 0:
                avg_l2 = sum(l_l2) / len(l_l2)
            else:
                avg_l2 = 0

            self.group_wealth_l = avg_h
            self.group_wealth_h = avg_h

            if typ == 'f':
                avg_l1 = self.model.ff_cost*avg_l1
                avg_l2 = self.model.nf_cost*avg_l2
            elif typ == 'm':
                avg_l1 = self.model.fm_cost*avg_l1
                avg_l2 = self.model.nm_cost*avg_l2

            l1 = [l_type1, l_type2, typ]
            l2 = [avg_l1, avg_l2, avg_h]

            self.type = [l1[i] for i in range(3) if l2[i] == max(l2)][0]


        num_fem = len([i for i in self.model.schedule.agents if len(i.type) > 1 and i.type.startswith('f')])
        self.model.fem_percent = num_fem / self.model.num_agents
        self.model.ff_percent = len([a for a in self.model.schedule.agents if a.type == 'ff']) / len([a for a in self.model.schedule.agents if a.type.endswith('f')])
        self.model.fm_percent = len([a for a in self.model.schedule.agents if a.type == 'fm']) / len([a for a in self.model.schedule.agents if a.type.endswith('m')])
        self.model.nf_percent = len([a for a in self.model.schedule.agents if a.type == 'nf']) / len([a for a in self.model.schedule.agents if a.type.endswith('f')])
        self.model.nm_percent = len([a for a in self.model.schedule.agents if a.type == 'nm']) / len([a for a in self.model.schedule.agents if a.type.endswith('m')])

    def identify(self, other, times):
        iden = False
        if len([i for i in self.met_history if i == set([self.unique_id, other.unique_id])]) >= times:
            iden = True
        return iden

    def gen_proporation(self,other, times,lb,ub, rate, tax_res):
        w0 = 0
        prp = 0
        if self.age == 't':
            s = sum([0.39, 0.09, 0.08, 0.07, 0.05, 0.17, 0.04,0.02,0.01,0.005, 0.026])
            rand_age = np.random.choice(rang, p=[i/s for i in [0.39, 0.09, 0.08, 0.07, 0.05, 0.17, 0.04,0.02,0.01,0.005, 0.026]])
        elif self.age == 'm':
            s = sum([0.1, 0.02, 0.02, 0.16, 0.01, 0.5, 0.18,0.015,0.005,0.002,0.026])
            rand_age = np.random.choice(rang, p=[i/s for i in [0.1, 0.02, 0.02, 0.16, 0.01, 0.5, 0.18,0.015,0.005,0.002,0.026]])
        elif self.age == 'o':
            s = sum([0,0,0,0,0.15, 0.3, 0.2,0.15,0.15,0.15,0.42])
            rand_age = np.random.choice(rang, p=[i/s for i in [0,0,0,0,0.15, 0.3, 0.2,0.15,0.15,0.15,0.42]])
        bench = rand_age
        if self.identify(other,times):
            s = sum([0.2, 0.1, 0.06, 0.065, 0.08, 0.39, 0.02,0.015,0.001,0.002, 0.022])
            rand_iden = np.random.choice(rang, p=[i/s for i in [0.2, 0.1, 0.06, 0.065, 0.08, 0.39, 0.02,0.015,0.001,0.002, 0.022]])
            r = random.uniform(lb,ub)
            bench = round(r*rand_iden + (1-r)*rand_age)
        if len(self.type) > 1:
            prp = gen_propo_type(self.type,other,bench, rate)
        else:
            if self.type == 'f':
                if w0!=0:
                    prp = other.model.ff_percent * gen_propo_type('ff',other,bench, rate) /w0 + other.model.nf_percent * gen_propo_type('nf',other,bench, rate)/w0
                else:
                    prp = random.uniform(gen_propo_type('ff',other,bench, rate), gen_propo_type('nf',other,bench, rate))
            if self.type == 'm':
                if w0!=0:
                    prp = other.model.fm_percent * gen_propo_type('fm',other,bench, rate) /w0 + other.model.nm_percent * gen_propo_type('nm',other,bench, rate)/w0
                else:
                    prp = random.uniform(gen_propo_type('fm',other,bench, rate), gen_propo_type('nm',other,bench,rate))

        if self.tax_rate != 0:
            if self.type in ['ff','fm'] and other.type in ['ff','fm']:
                prp = prp * (1 + tax_res*self.tax_rate)
            elif self.type == 'f':
                prp = prp*(1 + tax_res*self.tax_rate*self.model.ff_percent)
            elif self.type == 'm':
                prp = prp*(1 + tax_res*self.tax_rate*self.model.fm_percent)

        return prp

    def tax(self):
        rate = 0
        if self.type == 'ff':
            rate = random.uniform(0.1,0.2)
        elif self.type == 'fm':
            rate = random.uniform(0.05,0.1)
        self.tax_rate = rate
        self.wealth  = round(self.wealth * (1 - rate))


class Model(Model):
    """A model with some number of agents."""
    def __init__(self, N,percentage,times,lb,ub,rate,tax_res,ff_cost,fm_cost,nf_cost,nm_cost,tax,pie):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.running = True
        self.giving_df = pd.DataFrame(columns=['round','setting','fem_percent','id','wealth','age',
                                               'low group wealth', 'high group wealth',
                                               'init_type','type','other type','giving',
                                               'propo', 'ff_percent','fm_percent','nf_percent',
                                              'nm_percent'])
        self.lst_types = gen_type_lst(N)
        self.index = list(range(N))
        self.ages = gen_age_group(N)
        self.percentage = percentage
        self.times = times
        self.lb = lb
        self.ub = ub
        self.rate = rate
        self.init_rate = rate
        self.tax_res = tax_res
        self.ff_cost = ff_cost
        self.fm_cost = fm_cost
        self.nf_cost = nf_cost
        self.nm_cost = nm_cost
        self.tax = tax
        self.pie = pie
        # Create agents
        num_fem = 0
        num_ff = 0
        num_fm = 0
        for i in range(self.num_agents):
            a = Agent(i, self)
            self.schedule.add(a) #seems like a list of all agents
            if len(a.type) == 2 and a.type.startswith('f'):
                num_fem += 1
                if a.type == 'ff':
                    num_ff += 1
                else:
                    num_fm += 1
        self.fem_percent = num_fem / self.num_agents
        self.ff_percent = num_ff / len([a for a in self.schedule.agents if a.type.endswith('f')])
        self.fm_percent = num_fm / len([a for a in self.schedule.agents if a.type.endswith('m')])
        self.nf_percent = len([a for a in self.schedule.agents if a.type == 'nf']) / len([a for a in self.schedule.agents if a.type.endswith('f')])
        self.nm_percent = len([a for a in self.schedule.agents if a.type == 'nm']) / len([a for a in self.schedule.agents if a.type.endswith('m')])
        self.round = 0



    def step(self,i):
        '''Advance the model by one step.'''
        self.round = i
        self.schedule.step()


In [4]:
pct = [0.5,0.75]
times = [5,10]
lb = [0.5] #assuming equal weights on identification
rates = [0.065] #assuming mid-level discrimination
tax_res = [5,10,20] #2 tried no use
fu_df = []
fm_c = [1,0.4,0.8]
tax = [False,True]

for p in pct:
    for t in times:
        for ta in tax:
            if ta == True:
                for tr in tax_res:
                    m = Model(20,p,t,0.5,0.5,0.4,tr,1,1,1,1,ta,10)
                    for r in range(20):
                        m.step(r)
                    fu_df.append(m.giving_df)
            else:
                m = Model(20,p,t,0.5,0.5,0.4,1,1,1,1,1,ta,10)
                for r in range(20):
                    m.step(r)
                    fu_df.append(m.giving_df)


In [6]:
df = pd.concat(fu_df)

## Transaction Design
- if run into error saying 'cannot choose from empty sequence', reloading the notebook and rerunning the cosde will solve the bug

In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import random
import pandas as pd
import numpy as np


In [2]:

rang = [x / 10.0 for x in range(0, 11)]

def plus_to_param(num_p = 0, num_n = 0, r = 0):
    param = (1+r)**num_p*(1-r)**num_n
    return param

def gen_type_lst(num_agent):
    n_f= round(num_agent * 0.48)
    n_m = num_agent - n_f
    lst_types = []
    types = ['ff','nf','fm','nm'] #feminist female, non-feminist female, feminist male, non-feminist male

    num_ff = round(n_f * 0.47) #feminist female; data from statista (self report bias)
    lst_ff = ['ff'][:]*num_ff
    lst_types += lst_ff

    num_nf = n_f - num_ff
    lst_nf = ['nf'][:]*num_nf
    lst_types += lst_nf

    num_fm = round(n_m * 0.31) #feminist male; data from statista
    lst_fm = ['fm'][:]*num_fm
    lst_types += lst_fm

    num_nm = n_m - num_fm
    lst_nm = ['nm'][:]*num_nm
    lst_types += lst_nm

    return lst_types

def gen_age_group(l):
    s = sum([0.237, 0.524,0.126])
    l_teen = round(l * 0.237 / s)
    l_mid = round(l * 0.524 / s)
    l_old = l - l_teen - l_mid #data from statistica; not super precise; normalized to exclude children

    ages = ['t'][:]*l_teen + ['m'][:]*l_mid + ['o'][:]*l_old
    return ages

def gen_propo_type(typ,other,bench, rate):
    w = 0
    propo = bench
    if typ.startswith('f') and len(typ) > 1:
        if typ == 'ff':
            if other.type == 'ff':
                propo = bench * plus_to_param(num_p = 2, r = rate)
            elif other.type == 'nf':
                propo = bench * plus_to_param(num_n = 1, r = rate)
            elif other.type == 'nm':
                propo = bench * plus_to_param(num_n = 1, r = rate)
            elif other.type == 'f':
                w = other.model.ff_percent + other.model.nf_percent
                if w!=0:
                    propo = other.model.ff_percent * bench * plus_to_param(num_p = 2, r = rate) /w + other.model.nf_percent * bench * plus_to_param(num_n = 1, r = rate)/w
            elif other.type == 'm':
                w = other.model.fm_percent + other.model.nm_percent
                if w!=0:
                    propo = other.model.fm_percent * bench/w + other.model.nm_percent * bench* plus_to_param(num_n = 1, r = rate)/w
            else:
                propo = bench

        elif typ == 'fm':
            if other.type == 'ff':
                propo = bench * plus_to_param(num_p = 1, r = rate)
            elif other.type == 'fm':
                propo = bench * plus_to_param(num_p = 1, r = rate)
            elif other.type == 'nf':
                propo = bench * plus_to_param(num_p = 0, r = rate)
            elif other.type == 'nm':
                propo = bench * plus_to_param(num_n = 2, r = rate)
            elif other.type == 'f':
                w = other.model.ff_percent + other.model.nf_percent
                if w!=0:
                    propo = other.model.ff_percent * bench * plus_to_param(num_p = 1, r = rate) /w + other.model.nf_percent * bench * plus_to_param(num_p = 0, r = rate)/w
            elif other.type == 'm':
                w = other.model.fm_percent + other.model.nm_percent
                if w!=0:
                    propo = other.model.fm_percent * bench * plus_to_param(num_p = 1, r = rate) /w + other.model.nm_percent * bench * plus_to_param(num_n = 2, r = rate) /w

    elif typ.startswith('n') and len(typ) > 1:
        if typ == 'nf':
            if other.type == 'ff':
                propo = bench * plus_to_param(num_n = 3, r = rate)
            elif other.type == 'fm':
                propo = bench * plus_to_param(num_n = 0.5, r = rate)
            elif other.type == 'nf':
                propo = bench * plus_to_param(num_p = 0, r = rate)
            elif other.type == 'nm':
                propo = bench * plus_to_param(num_p = 0.5, r = rate)
            elif other.type == 'f':
                w = other.model.ff_percent + other.model.nf_percent
                if w!=0:
                    propo = other.model.ff_percent * bench * plus_to_param(num_n = 3, r = rate) /w + other.model.nf_percent * bench * plus_to_param(num_p = 0, r = rate)/w
            elif other.type == 'm':
                w = other.model.fm_percent + other.model.nm_percent
                if w!=0:
                    propo = other.model.fm_percent * bench * plus_to_param(num_n = 0.5, r = rate) /w + other.model.nm_percent * bench * plus_to_param(num_p = 0.5, r = rate) /w
        elif typ == 'nm':
            if other.type == 'ff':
                propo = bench * plus_to_param(num_n = 1.5, r = rate)
            elif other.type == 'fm':
                propo = bench * plus_to_param(num_n = 1.5, r = rate)
            elif other.type == 'nf':
                propo = bench * plus_to_param(num_n = 0.5, r = rate)
            elif other.type == 'nm':
                propo = bench * plus_to_param(num_p = 1.5, r = rate)
            elif other.type == 'f':
                w = other.model.ff_percent + other.model.nf_percent
                if w!=0:
                    propo = other.model.ff_percent * bench * plus_to_param(num_n = 1.5, r = rate) /w + other.model.nf_percent * bench *plus_to_param(num_n = 0.5, r = rate) /w
            elif other.type == 'm':
                w = other.model.fm_percent + other.model.nm_percent
                if w!=0:
                    propo = other.model.fm_percent * bench *plus_to_param(num_n = 1.5, r = rate) /w + other.model.nm_percent * bench * plus_to_param(num_p = 1.5, r = rate) /w


    return propo

class Agent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        r = random.choice(self.model.index)
        rand = self.model.lst_types[r]
        self.type = rand
        self.init_type = rand
        self.age = self.model.ages[r]
        if self.unique_id not in self.model.giving_df['id']:
            if self.init_type == 'fm':
                self.wealth = 10
            elif self.init_type == 'ff':
                self.wealth = 8
            elif self.init_type == 'nm':
                self.wealth = 5
            else:
                self.wealth = 1
        else:
            df = self.model.giving_df
            self.wealth = list(df['wealth'][df['id'] == self.unique_id])[-1]
        self.model.index.remove(r)
        self.met = [self.unique_id]
        self.tax_rate = 0
        self.group_wealth_l = 0
        self.group_wealth_h = 0
        self.met_history = []
        c = random.uniform(0,1)
        if c < ((0.3+0.31+0.54+0.59+0.46)/5):
            alpha = random.uniform(0,0.4)
        elif (c - (0.3+0.31+0.54+0.59+0.46)/5) < ((0.3+0.33+0.18+0.12+0.17)/5):
            alpha = random.uniform(0.4,0.92)
        elif (c - (0.3+0.31+0.54+0.59+0.46)/5 - (0.3+0.33+0.18+0.12+0.17)/5) < ((0.3+0.23+0.21+0.05+0.2)/5):
            alpha = random.uniform(0.92,4.5)
        else:
            alpha = random.uniform(4.5,9)
        alpha = round(alpha,2)
        self.alpha = alpha

        c = random.uniform(0,1)
        if c < ((0.3+0.29+0.21+0.16+0.20)/5):
            beta = random.uniform(0,0.235)
        elif (c - (0.3+0.29+0.21+0.16+0.20)/5) < ((0.3+0.15+0.25+0.11+0.19)/5):
            beta = random.uniform(0.235,0.5)
        elif (c - (0.3+0.31+0.54+0.59+0.46)/5 - (0.3+0.15+0.25+0.11+0.19)/5) < ((0.4+0.56+0.54+0.73+0.61)/5):
            beta = random.uniform(0.5,1)
        beta = round(beta,2)
        self.beta = beta
        



    def step(self):
        

        if len(self.model.giving_df) > 9 and list(self.model.giving_df['id'])[-1] != self.unique_id:
            self.met = [self.unique_id]

        while len(self.met) != round(len(self.model.schedule.agents) * self.model.percentage):
            if len([i for i in self.model.giving_df['id'] if i == self.unique_id]) >= 1:
                if self.model.tax == True:
                    self.tax()
                self.set_type()

                self.alpha = self.alpha*(1-self.model.fem_percent + (self.model.nm_percent + self.model.nf_percent)/2)
                self.beta = self.beta*(1-(self.model.nm_percent + self.model.nf_percent)/2 + self.model.fem_percent)

                if self.alpha < 0:
                    self.alpha = 0
                elif self.alpha > 9:
                    self.alpha = 9

                if self.beta < 0:
                    self.beta = 0
                elif self.beta > 1:
                    self.beta = 1


            self.group_wealth_l = sum([i.wealth for i in
                                       self.model.schedule.agents if
                                       i.type == self.type]) / len([i.wealth
                                                                    for i in self.model.schedule.agents
                                                                    if i.type == self.type])
            self.group_wealth_h = sum([i.wealth for i in
                                       self.model.schedule.agents if
                                       i.type[-1] == self.type[-1]]) / len([i
                                                                            for i in self.model.schedule.agents
                                                                            if i.type[-1] == self.type[-1]])
            other_agent = random.choice(self.model.schedule.agents)
            if other_agent.unique_id not in self.met:
                self.met_history.append(set([self.unique_id,other_agent.unique_id]))
                other_agent.met_history.append(set([self.unique_id,other_agent.unique_id]))

                s_c = round(random.uniform(0.01,0.4) * self.wealth,2)
                s_p = s_c + round(random.uniform(0,0.6) * s_c,2)

                o_c = round(random.uniform(0.01,0.4) * other_agent.wealth,2)
                o_p = o_c + round(random.uniform(0,0.6) * o_c,2)

                s_b = (random.uniform(0,2) - 1) * o_p 
                o_b = (random.uniform(0,2) - 1) * s_p
                
                s_bg = 0
                s_be = 0
                s_ng = 0
                s_ne = 0
                o_bg = 0
                o_be = 0
                o_ng = 0
                o_ne = 0

                propo = gen_propo_type(self.type,other_agent,0.5,self.model.rate) # the closer, the higher
                if self.tax_rate != 0:
                    if self.type in ['ff','fm'] and other_agent.type in ['ff','fm']:
                        propo = propo * (1 + self.model.tax_res*self.tax_rate)
                    elif self.type == 'f'and other_agent.type in ['ff','fm']:
                        propo = propo*(1 + self.model.tax_res*self.tax_rate*self.model.ff_percent)
                    elif self.type == 'm'and other_agent.type in ['ff','fm']:
                        propo = propo*(1 + self.model.tax_res*self.tax_rate*self.model.fm_percent)
                if propo > 1:
                    propo = 1

                s_alpha = (1-propo)*self.alpha*2
                s_beta = propo*self.beta*2

                propo = gen_propo_type(other_agent.type,self,0.5, self.model.rate) # the closer, the higher
                if other_agent.tax_rate != 0:
                    if other_agent.type in ['ff','fm'] and other_agent.type in ['ff','fm']:
                        propo = propo * (1 + other_agent.model.tax_res*other_agent.tax_rate)
                    elif other_agent.type == 'f'and other_agent.type in ['ff','fm']:
                        propo = propo*(1 + other_agent.model.tax_res*other_agent.tax_rate*other_agent.model.ff_percent)
                    elif other_agent.type == 'm'and other_agent.type in ['ff','fm']:
                        propo = propo*(1 + other_agent.model.tax_res*other_agent.tax_rate*other_agent.model.fm_percent)
                if propo > 1:
                    propo = 1

                other_agent.alpha = other_agent.alpha*(1-other_agent.model.fem_percent + (other_agent.model.nm_percent + other_agent.model.nf_percent)/2)
                other_agent.beta = other_agent.beta*(1-(other_agent.model.nm_percent + other_agent.model.nf_percent)/2 + other_agent.model.fem_percent)
                if other_agent.alpha < 0:
                    other_agent.alpha = 0
                elif other_agent.alpha > 9:
                    other_agent.alpha = 9

                if other_agent.beta < 0:
                    other_agent.beta = 0
                elif other_agent.beta > 1:
                    other_agent.beta = 1
                
                o_alpha = (1-propo)*other_agent.alpha*2
                o_beta = propo*other_agent.beta*2
                

                
                if self.wealth == 0:
                    if (self.wealth + s_b - other_agent.wealth - (o_p - o_c)) > 0:
                        s_bg = 1
                    elif (other_agent.wealth + (o_p - o_c) - (self.wealth + s_b)) > 0:
                        s_be = 1
                else:
                    s_bg = max([(self.wealth + s_b - other_agent.wealth - (o_p - o_c))/(self.wealth+s_b),0])
                    s_be = max([(other_agent.wealth + (o_p - o_c) - (self.wealth + s_b))/(self.wealth+s_b),0])
                    s_ng = max([(self.wealth - other_agent.wealth)/self.wealth,0])
                    s_ne = max([(other_agent.wealth - self.wealth)/self.wealth,0])

                if other_agent.wealth == 0:
                    if (other_agent.wealth + s_b - self.wealth - (o_p - o_c)) > 0:
                        o_bg = 1
                    elif (self.wealth + (o_p - o_c) - (other_agent.wealth + s_b)) > 0:
                        o_be = 1
                else:
                    o_bg = max([(other_agent.wealth + s_b - self.wealth - (o_p - o_c))/(other_agent.wealth+s_b),0])
                    o_be = max([(self.wealth + (o_p - o_c) - (other_agent.wealth + s_b))/(other_agent.wealth+s_b),0])
                    o_ng = max([(other_agent.wealth - self.wealth)/other_agent.wealth,0])
                    o_ne = max([(self.wealth - other_agent.wealth)/other_agent.wealth,0])

                s_bu = self.wealth + s_b - round(s_alpha*10*s_be, 2)- round(s_beta*10*s_bg, 2)
                s_nu = self.wealth - round(s_alpha*10*s_ne,2) - round(s_beta*10*s_ng,2)

                o_bu = other_agent.wealth + o_b - round(o_alpha*10*o_be,2) - round(o_beta*10*o_bg,2)
                o_nu = other_agent.wealth - round(o_alpha*10*o_ne,2) - round(o_beta*10*o_ng,2)
                
                if s_bu > s_nu and o_p <= self.wealth:
                    self.wealth += s_b
                    other_agent.wealth += (o_p - o_c)

                if o_bu > o_nu and s_p <= other_agent.wealth:
                    other_agent.wealth += o_b
                    self.wealth += (s_p - s_c)
                if self.wealth < 0:
                    self.wealth = 0
                if other_agent.wealth<0:
                    other_agent.wealth=0

                self.met.append(other_agent.unique_id)
                setting = str(self.model.percentage)+'|'+str(self.model.tax_res) + '|' + str(self.model.fm_cost)
                





                self.model.giving_df.loc[len(self.model.giving_df.index)] = [self.model.round,setting,self.model.fem_percent,
                                                                             self.unique_id, self.wealth,
                                                                             self.age,self.group_wealth_l, self.group_wealth_h,
                                                                             self.init_type, self.type, other_agent.type,
                                                                             self.model.ff_percent, self.model.fm_percent,
                                                                             self.model.nf_percent,self.model.nm_percent,
                                                                             self.alpha,self.beta]

    def set_type(self):
        typ = self.type
        if len(typ) == 2:
            if typ in ['ff','nf']:
                h_type = 'f'
                ol_type = [i for i in ['ff','nf'] if i != typ][0]
            else:
                h_type = 'm'
                ol_type = [i for i in ['fm','nm'] if i != typ][0]


            l_h = [i.wealth for i in self.model.schedule.agents if i.type[-1] == typ[-1]]
            avg_h = sum(l_h) / len(l_h)

            l_l = [i.wealth for i in self.model.schedule.agents if i.type == typ]
            if len(l_l) != 0:
                avg_l = sum(l_l) / len(l_l)
            else:
                avg_l = 0

            l_ol = [i.wealth for i in self.model.schedule.agents if i.type == ol_type]
            if len(l_ol) != 0:
                avg_ol = sum(l_ol) / len(l_ol)
            else:
                avg_ol = 0


            self.group_wealth_l = avg_l
            self.group_wealth_h = avg_h

            if typ == 'ff':
                avg_l = self.model.ff_cost*avg_l
                avg_ol = self.model.nf_cost*avg_ol
            elif typ == 'nf':
                avg_l = self.model.nf_cost*avg_l
                avg_ol = self.model.ff_cost*avg_ol
            elif typ == 'fm':
                avg_l = self.model.fm_cost*avg_l
                avg_ol = self.model.nm_cost*avg_ol
            elif typ == 'nm':
                avg_l = self.model.nm_cost*avg_l
                avg_ol = self.model.fm_cost*avg_ol


            l1 = [typ, h_type, ol_type]
            l2 = [avg_l, avg_h, avg_ol]

            self.type = [l1[i] for i in range(3) if l2[i] == max(l2)][0]

        else:
            l_type1 = 'f' + typ
            l_type2 = 'n' + typ

            l_h = [i.wealth for i in self.model.schedule.agents if i.type == typ]
            avg_h = sum(l_h) / len(l_h)

            l_l1 = [i.wealth for i in self.model.schedule.agents if i.type == l_type1]
            if len(l_l1) != 0:
                avg_l1 = sum(l_l1) / len(l_l1)
            else:
                avg_l1 = 0

            l_l2 = [i.wealth for i in self.model.schedule.agents if i.type == l_type2]
            if len(l_l2) != 0:
                avg_l2 = sum(l_l2) / len(l_l2)
            else:
                avg_l2 = 0

            self.group_wealth_l = avg_h
            self.group_wealth_h = avg_h

            if typ == 'f':
                avg_l1 = self.model.ff_cost*avg_l1
                avg_l2 = self.model.nf_cost*avg_l2
            elif typ == 'm':
                avg_l1 = self.model.fm_cost*avg_l1
                avg_l2 = self.model.nm_cost*avg_l2

            l1 = [l_type1, l_type2, typ]
            l2 = [avg_l1, avg_l2, avg_h]

            self.type = [l1[i] for i in range(3) if l2[i] == max(l2)][0]


        num_fem = len([i for i in self.model.schedule.agents if len(i.type) > 1 and i.type.startswith('f')])
        self.model.fem_percent = num_fem / self.model.num_agents
        self.model.ff_percent = len([a for a in self.model.schedule.agents if a.type == 'ff']) / len([a for a in self.model.schedule.agents if a.type.endswith('f')])
        self.model.fm_percent = len([a for a in self.model.schedule.agents if a.type == 'fm']) / len([a for a in self.model.schedule.agents if a.type.endswith('m')])
        self.model.nf_percent = len([a for a in self.model.schedule.agents if a.type == 'nf']) / len([a for a in self.model.schedule.agents if a.type.endswith('f')])
        self.model.nm_percent = len([a for a in self.model.schedule.agents if a.type == 'nm']) / len([a for a in self.model.schedule.agents if a.type.endswith('m')])

    def identify(self, other, times):
        iden = False
        if len([i for i in self.met_history if i == set([self.unique_id, other.unique_id])]) >= times:
            iden = True
        return iden


    def tax(self):
        rate = 0
        if self.type == 'ff':
            rate = random.uniform(0.1,0.2)
        elif self.type == 'fm':
            rate = random.uniform(0.05,0.1)
        self.tax_rate = rate
        self.wealth  = round(self.wealth * (1 - rate))


class Model(Model):
    """A model with some number of agents."""
    def __init__(self, N,percentage,rate,tax_res,ff_cost,fm_cost,nf_cost,nm_cost, tax):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.running = True
        self.giving_df = pd.DataFrame(columns=['round','setting','fem_percent','id','wealth','age',
                                               'low group wealth', 'high group wealth',
                                               'init_type','type','other type','ff_percent','fm_percent',
                                               'nf_percent','nm_percent','alpha','beta'])
        self.lst_types = gen_type_lst(N)
        self.index = list(range(N))
        self.ages = gen_age_group(N)
        self.percentage = percentage
        self.rate = rate
        self.tax_res = tax_res
        self.ff_cost = ff_cost
        self.fm_cost = fm_cost
        self.nf_cost = nf_cost
        self.nm_cost = nm_cost
        self.tax = tax
        # Create agents
        num_fem = 0
        num_ff = 0
        num_fm = 0
        for i in range(self.num_agents):
            a = Agent(i, self)
            self.schedule.add(a) #seems like a list of all agents
            if len(a.type) == 2 and a.type.startswith('f'):
                num_fem += 1
                if a.type == 'ff':
                    num_ff += 1
                else:
                    num_fm += 1
        self.fem_percent = num_fem / self.num_agents
        self.ff_percent = num_ff / len([a for a in self.schedule.agents if a.type.endswith('f')])
        self.fm_percent = num_fm / len([a for a in self.schedule.agents if a.type.endswith('m')])
        self.nf_percent = len([a for a in self.schedule.agents if a.type == 'nf']) / len([a for a in self.schedule.agents if a.type.endswith('f')])
        self.nm_percent = len([a for a in self.schedule.agents if a.type == 'nm']) / len([a for a in self.schedule.agents if a.type.endswith('m')])
        self.round = 0



    def step(self,i):
        '''Advance the model by one step.'''
        self.round = i
        self.schedule.step()


In [ ]:
pct = [0.5,0.75]
tax_res = [5,10,20]
fu_df = []
fm_c = [1,0.4,0.8]
tax = [False,True]

for p in pct:
    for fm in fm_c:
        if fm == 1:
            ff = 1
            nf = 1
            nm = 1
        else:
            ff = fm + 0.05
            nf = fm + 0.1
            nm = fm + 0.15
        for ta in tax:
            if ta == True:
                for tr in tax_res:
                    m = Model(20,p,0.4,tr,ff,fm,nf,nm,ta)
                    for r in range(20):
                        m.step(r)
                    fu_df.append(m.giving_df)

            else:
                m = Model(20,p,0.4,1,ff,fm,nf,nm,ta)
                for r in range(20):
                    m.step(r)
                    fu_df.append(m.giving_df)
                        


In [4]:
df = pd.concat(fu_df)